### Pipiline

1. Preprocesamiento de documentos
Necesitas convertir PDFs de artículos científicos a texto
Dividir el contenido en chunks (fragmentos) manejables
Crear embeddings (representaciones vectoriales) de estos fragmentos
2. Base de datos vectorial
Almacena los embeddings para búsqueda eficiente
Permite encontrar fragmentos similares a una consulta
3. Pipeline de extracción de metadatos
Define qué metadatos quieres extraer (título, autores, abstract, palabras clave, etc.)
Usa el contexto recuperado para generar respuestas estructuradas

### Loading PDF

In [2]:
from langchain_community.document_loaders import PyPDFLoader
import pprint
import os

file_path = "/home/cristian/projects/rag_pae/data/pdfs/amazonica/A50.pdf"
doc = None

if os.path.isfile(file_path):
    try:
        if os.path.splitext(file_path)[1].lower() != '.pdf':
            raise ValueError("The file is not a PDF.")
        loader = PyPDFLoader(file_path)
        doc = loader.load()
        print("PDF loaded successfully.")
    except Exception as e:
        print(f"Error loading PDF: {e}")
else:
    print(f"File not found: {file_path}")

PDF loaded successfully.


In [3]:
pprint.pp(doc[1].page_content)


('Caravanas, migrantes y desplazados: \n'
 'experiencias y debates en torno a las formas contemporáneas de movilidad '
 'humana \n'
 ' \n'
 '175 \n'
 'Iberoforum. Revista de Ciencias Sociales de la Universidad Iberoamericana. \n'
 'Año XIV, No. 27, enero – junio 2019. \n'
 'Angela Yesenia Olaya Requene, pp. 175- 208, ISSN: 2007-0675 \n'
 'Universidad Iberoamericana Ciudad de México, www.ibero.mx/iberoforum/27 \n'
 ' \n'
 'LA FRONTERA ENTRE COLOMBIA Y ECUADOR:  \n'
 'MOVILIDADES DE COMUNIDADES AFROCOLOMBIANAS  \n'
 'EN ESCENARIOS DEL NARCOTRÁFICO \n'
 ' \n'
 'The Border Between Colombia and Ecuador: Mobilities of the Afro-Colombian '
 'Communities \n'
 ' in Drug Trafficking Contexts \n'
 ' \n'
 ' \n'
 'Angela Yesenia Olaya Requene \n'
 ' \n'
 ' \n'
 'Resumen \n'
 'l artículo analiza las dinámicas de movilidad \n'
 'local/trasnacional de comunidades \n'
 'afrocolombianas en la frontera entre Colombia y \n'
 'Ecuador por el Pacífico sur colombiano. Se describen los \n'
 'desplazamientos y

### Chunking doc

In [4]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding= embeddings)
retriever = vectorstore.as_retriever()


In [20]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# Inicializar embeddings para el semantic chunker
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Crear el semantic chunker
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="percentile",  # Opciones: "percentile", "standard_deviation", "interquartile"
    breakpoint_threshold_amount=95,  # Percentil 95 para encontrar breakpoints
    number_of_chunks=None,  # Déjalo None para división automática
    buffer_size=50  # Número de oraciones a considerar para el contexto
)

# Dividir el documento usando semantic chunking
semantic_splits = semantic_splitter.split_documents(doc)

print(f"Número de chunks semánticos: {len(semantic_splits)}")

# Crear vectorstore con semantic chunks
vectorstore = Chroma.from_documents(
    documents=semantic_splits, 
    embedding=embeddings,
    collection_name="semantic_chunks"
)
retriever = vectorstore.as_retriever()

Número de chunks semánticos: 39


In [5]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from datetime import date
from enum import Enum

class DocumentType(str, Enum):
    ARTICLE = "article"
    REVIEW = "review"
    BOOK_CHAPTER = "book_chapter"
    CONFERENCE_PAPER = "conference_paper"
    THESIS = "thesis"
    REPORT = "report"
    OTHER = "other"

class Language(str, Enum):
    SPANISH = "es"
    ENGLISH = "en"
    PORTUGUESE = "pt"
    FRENCH = "fr"
    OTHER = "other"

class Region(str, Enum):
    AMAZONIA = "amazonia"
    CARIBE = "caribe"
    PACIFICO = "pacifico"
    ANDINA = "andina"
    ORINOQUIA = "orinoquia"
    OTHER = "other"

class ScienceDirectDocument(BaseModel):
    # Metadatos básicos
    authors: List[str] = Field(default_factory=list, description="Lista de autores del documento", alias="autor")
    typology: str = Field(default="", description="Tipología del documento", alias="tipologia")
    year: int = Field(default=1900, description="Año de publicación", ge=1900, le=2030, alias="año")
    title: str = Field(default="", description="Título del documento", alias="titulo")
    
    # Contenido y análisis
    objective: str = Field(default="", description="Objetivo al que responde el documento")
    relevance: str = Field(default="", description="Relevancia del documento para el estudio")
    document_type: DocumentType = Field(default=DocumentType.OTHER, description="Tipo de documento")
    gis_information: str = Field(default="", description="Información SIG (Sistemas de Información Geográfica)")
    language: Language = Field(default=Language.SPANISH, description="Idioma del documento")
    keywords: List[str] = Field(default_factory=list, description="Palabras clave del documento")
    
    # Referencias y ubicación
    apa_reference: str = Field(default="", description="Referencia en formato APA")
    pages: str = Field(default="", description="Número de páginas or rango de páginas")
    abstract: str = Field(default="", description="Resumen del documento")
    
    # Análisis por región
    regional_review_comments: str = Field(default="", description="Comentario de revisión por región")
    relevant_pages_by_region: str = Field(default="", description="Páginas relevantes por región")
    thesaurus_categories: List[str] = Field(default_factory=list, description="Categorías del tesauro")
    
    # Análisis de victimización y contexto
    victimizing_event: str = Field(default="", description="Hecho victimizante identificado")
    damage: str = Field(default="", description="Daño o impacto identificado")
    actor: str = Field(default="", description="Actor involucrado")
    location: str = Field(default="", description="Lugar general")
    specific_location_with_review: str = Field(default="", description="Lugar específico con revisión")
    context_year: int = Field(default=1900, description="Año del contexto analizado", ge=1900, le=2030)
    region: Region = Field(default=Region.OTHER, description="Región geográfica")
    
    # Metadatos técnicos adicionales (EXTRA - no requeridos pero útiles)
    doi: str = Field(default="", description="DOI del documento")
    journal: str = Field(default="", description="Revista o fuente de publicación")

    model_config = {
        "use_enum_values": True,
        "populate_by_name": True  # V2 equivalent of allow_population_by_field_name
    }
        
    @field_validator('year', 'context_year')
    @classmethod
    def validate_years(cls, v):
        """Validar que los años estén en rango razonable"""
        if v < 1900 or v > 2030:
            return 1900  # Valor por defecto si está fuera de rango
        return v
    
    @field_validator('authors', 'keywords', 'thesaurus_categories')
    @classmethod
    def validate_lists(cls, v):
        """Limpiar listas de strings vacíos"""
        if isinstance(v, list):
            return [item.strip() for item in v if item and item.strip()]
        return v

In [1]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2", temperature=0.1)

In [8]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.prompts import PromptTemplate
import json

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


schema = ScienceDirectDocument.model_json_schema()

system_prompt = """You are a specialized document metadata extractor for academic documents.

EXTRACT THE FOLLOWING FIELDS FROM THE DOCUMENT:

BASIC METADATA:
- authors: All author names as array of strings
- typology: Document typology/classification
- year: Publication year (integer, 1900-2030)
- title: Complete document title

CONTENT ANALYSIS:
- objective: Research objective or purpose
- relevance: Document relevance for the study
- document_type: "article", "review", "book_chapter", "conference_paper", "thesis", "report", "other"
- gis_information: GIS/Geographic information systems data
- language: "es", "en", "pt", "fr", "other"
- keywords: Key terms and phrases as array
- abstract: Document summary/abstract
- pages: Page numbers or range

REGIONAL ANALYSIS:
- regional_review_comments: Regional review comments
- relevant_pages_by_region: Relevant pages per region
- thesaurus_categories: Thesaurus categories as array
- region: "amazonia", "caribe", "pacifico", "andina", "orinoquia", "other"

VICTIMIZATION CONTEXT:
- victimizing_event: Identified victimizing events
- damage: Identified damage or impact
- actor: Involved actors
- location: General location
- specific_location_with_review: Specific location with review
- context_year: Context year (integer, 1900-2030)

REFERENCES:
- apa_reference: Complete APA citation

INSTRUCTIONS:
1. Extract ONLY information explicitly found in the document
2. Use default values for missing data
3. Output valid JSON without explanations
4. Follow enum restrictions exactly

CONTEXT:
{context}

JSON OUTPUT:"""

prompt = PromptTemplate.from_template (
    system_prompt)


rag_chain = (
    {"context": retriever | format_docs, "schema": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

output = (rag_chain.invoke(str(schema)))
print("Output:")
pprint.pprint((output))


Output:
('{\n'
 '  "basic_metadata": {\n'
 '    "authors": [\n'
 '      "Angela Yesenia Olaya Requene"\n'
 '    ],\n'
 '    "typology": "article",\n'
 '    "year": 2019,\n'
 '    "title": "La frontera entre Colombia y Ecuador: Movilidades de '
 'comunidades afrocolombianas en escenarios del narcotráfico"\n'
 '  },\n'
 '  "content_analysis": {\n'
 '    "objective": "Investigar las movilidades de comunidades afrocolombianas '
 'en escenarios del narcotráfico",\n'
 '    "relevance": "Relevante para el estudio de la migración y la violencia '
 'en América Latina",\n'
 '    "document_type": "article",\n'
 '    "gis_information": null,\n'
 '    "language": "es",\n'
 '    "keywords": [\n'
 '      "migración",\n'
 '      "violencia",\n'
 '      "narcotráfico",\n'
 '      "comunidades afrocolombianas"\n'
 '    ],\n'
 '    "abstract": "Investiga las movilidades de comunidades afrocolombianas en '
 'escenarios del narcotráfico",\n'
 '    "pages": "pp. 175-208"\n'
 '  },\n'
 '  "regional_analysis"

In [146]:
vectorstore.delete_collection()